In [180]:
!pip install textblob

In [181]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

In [182]:
playlist_df = pd.read_csv("../spotifyrecommendationsystem/base_data.csv")
print(playlist_df.columns)
playlist_df.drop(columns=["Unnamed: 0"], inplace = True)
playlist_df.head()

Index(['Unnamed: 0', 'artist', 'album', 'track_name', 'track_id',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'artist_pop', 'genres', 'track_pop'],
      dtype='object')


,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,analysis_url,duration_ms,time_signature,artist_pop,genres,track_pop
0,ROLE MODEL,forever&more,forever&more,0UMFm4ndFr3PH5VsVYnCNG,0.694,0.637,9,-5.896,1,0.0806,0.298,0.000010,0.1230,0.506,83.982,https://api.spotify.com/v1/tracks/0UMFm4ndFr3P...,https://api.spotify.com/v1/audio-analysis/0UMF...,175609,4,56,alt_z bedroom_pop pov:_indie,0
1,ROLE MODEL,"oh, how perfect",hello!,7a2IJHzw9WJjoknwdnCop0,0.791,0.462,10,-4.023,1,0.0383,0.505,0.000000,0.0991,0.556,125.913,https://api.spotify.com/v1/tracks/7a2IJHzw9WJj...,https://api.spotify.com/v1/audio-analysis/7a2I...,188547,4,56,alt_z bedroom_pop pov:_indie,57
2,ROLE MODEL,"oh, how perfect",gucci chair,0GD2v2Gl24z7F6bLl4fn7M,0.368,0.469,1,-7.800,0,0.0346,0.304,0.016400,0.1070,0.129,150.447,https://api.spotify.com/v1/tracks/0GD2v2Gl24z7...,https://api.spotify.com/v1/audio-analysis/0GD2...,187613,3,56,alt_z bedroom_pop pov:_indie,36
3,ROLE MODEL,"oh, how perfect",that's just how it goes,4J4rJEaVpgr2tHwX6Jd13H,0.778,0.629,7,-5.011,0,0.0310,0.189,0.000293,0.1490,0.480,108.023,https://api.spotify.com/v1/tracks/4J4rJEaVpgr2...,https://api.spotify.com/v1/audio-analysis/4J4r...,166400,4,56,alt_z bedroom_pop pov:_indie,53
4,ROLE MODEL,"oh, how perfect",notice me (feat. BENEE),3WowsoNoWeMqTH2suWv9MW,0.720,0.481,8,-6.516,0,0.0432,0.136,0.003400,0.3010,0.253,126.005,https://api.spotify.com/v1/tracks/3WowsoNoWeMq...,https://api.spotify.com/v1/audio-analysis/3Wow...,191880,4,56,alt_z bedroom_pop pov:_indie,55


In [183]:
duplicates = playlist_df.duplicated()
num_duplicates = duplicates.sum()
num_null = df.isnull().sum()
print(num_duplicates)
print(num_null)

0
Unnamed: 0     0
pos            0
artist_name    0
track_uri      0
artist_uri     0
track_name     0
album_uri      0
duration_ms    0
album_name     0
name           0
dtype: int64


In [184]:
columns_to_drop = ["album", "track_href", "analysis_url", "duration_ms", "time_signature"]
playlist_df = playlist_df.drop(columns=columns_to_drop)

In [185]:
playlist_df.columns

Index(['artist', 'track_name', 'track_id', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'artist_pop', 'genres', 'track_pop'],
      dtype='object')

In [186]:
def preprocess_genre(df):
    df['genres_list'] = df['genres'].apply(lambda x: x.split(" "))
    return df

playlist_df = preprocess_genre(playlist_df)
playlist_df['genres_list'].head()

0    [alt_z, bedroom_pop, pov:_indie]
1    [alt_z, bedroom_pop, pov:_indie]
2    [alt_z, bedroom_pop, pov:_indie]
3    [alt_z, bedroom_pop, pov:_indie]
4    [alt_z, bedroom_pop, pov:_indie]
Name: genres_list, dtype: object

In [187]:
playlist_df

,artist,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list
0,ROLE MODEL,forever&more,0UMFm4ndFr3PH5VsVYnCNG,0.694,0.637,9,-5.896,1,0.0806,0.2980,0.000010,0.1230,0.506,83.982,56,alt_z bedroom_pop pov:_indie,0,"[alt_z, bedroom_pop, pov:_indie]"
1,ROLE MODEL,hello!,7a2IJHzw9WJjoknwdnCop0,0.791,0.462,10,-4.023,1,0.0383,0.5050,0.000000,0.0991,0.556,125.913,56,alt_z bedroom_pop pov:_indie,57,"[alt_z, bedroom_pop, pov:_indie]"
2,ROLE MODEL,gucci chair,0GD2v2Gl24z7F6bLl4fn7M,0.368,0.469,1,-7.800,0,0.0346,0.3040,0.016400,0.1070,0.129,150.447,56,alt_z bedroom_pop pov:_indie,36,"[alt_z, bedroom_pop, pov:_indie]"
3,ROLE MODEL,that's just how it goes,4J4rJEaVpgr2tHwX6Jd13H,0.778,0.629,7,-5.011,0,0.0310,0.1890,0.000293,0.1490,0.480,108.023,56,alt_z bedroom_pop pov:_indie,53,"[alt_z, bedroom_pop, pov:_indie]"
4,ROLE MODEL,notice me (feat. BENEE),3WowsoNoWeMqTH2suWv9MW,0.720,0.481,8,-6.516,0,0.0432,0.1360,0.003400,0.3010,0.253,126.005,56,alt_z bedroom_pop pov:_indie,55,"[alt_z, bedroom_pop, pov:_indie]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,WOOSUNG,You Make Me Back,1cwHRnup3iQoGAzUDwXQsh,0.536,0.507,3,-6.204,0,0.0276,0.5540,0.000070,0.0599,0.344,110.056,58,k-pop korean_ost,6,"[k-pop, korean_ost]"
96,WOOSUNG,FACE,1cBdZdp3VuzMc0gsWSGQvy,0.954,0.356,6,-6.213,0,0.0967,0.0949,0.000000,0.0581,0.513,113.034,58,k-pop korean_ost,0,"[k-pop, korean_ost]"
97,WOOSUNG,Beautiful Girl,5y3iGBS4FnaAF75VF9It5b,0.744,0.771,1,-3.936,0,0.0732,0.4190,0.000000,0.1820,0.524,102.945,58,k-pop korean_ost,0,"[k-pop, korean_ost]"
98,Lorde,Team,3G6hD9B2ZHOsgf4WfNu7X1,0.690,0.578,6,-7.436,1,0.0929,0.1670,0.000000,0.3050,0.416,99.961,75,art_pop metropopolis nz_pop pop,80,"[art_pop, metropopolis, nz_pop, pop]"


In [188]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
    if task == "subjectivity":
        if score < 1/3:
            return "low"
        elif score > 1/3:
            return "high"
        else:
            return "medium"
    else:
        if score < 0:
            return 'negative'
        elif score == 0:
            return 'neutral'
        else:
            return 'positive'
        
def sentiment_analysis(df, text_col):
    df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
    df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
    return df

In [189]:
sentiment = sentiment_analysis(playlist_df, "track_name")
sentiment.head()

,artist,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,ROLE MODEL,forever&more,0UMFm4ndFr3PH5VsVYnCNG,0.694,0.637,9,-5.896,1,0.0806,0.298,0.000010,0.1230,0.506,83.982,56,alt_z bedroom_pop pov:_indie,0,"[alt_z, bedroom_pop, pov:_indie]",low,neutral
1,ROLE MODEL,hello!,7a2IJHzw9WJjoknwdnCop0,0.791,0.462,10,-4.023,1,0.0383,0.505,0.000000,0.0991,0.556,125.913,56,alt_z bedroom_pop pov:_indie,57,"[alt_z, bedroom_pop, pov:_indie]",low,neutral
2,ROLE MODEL,gucci chair,0GD2v2Gl24z7F6bLl4fn7M,0.368,0.469,1,-7.800,0,0.0346,0.304,0.016400,0.1070,0.129,150.447,56,alt_z bedroom_pop pov:_indie,36,"[alt_z, bedroom_pop, pov:_indie]",low,neutral
3,ROLE MODEL,that's just how it goes,4J4rJEaVpgr2tHwX6Jd13H,0.778,0.629,7,-5.011,0,0.0310,0.189,0.000293,0.1490,0.480,108.023,56,alt_z bedroom_pop pov:_indie,53,"[alt_z, bedroom_pop, pov:_indie]",low,neutral
4,ROLE MODEL,notice me (feat. BENEE),3WowsoNoWeMqTH2suWv9MW,0.720,0.481,8,-6.516,0,0.0432,0.136,0.003400,0.3010,0.253,126.005,56,alt_z bedroom_pop pov:_indie,55,"[alt_z, bedroom_pop, pov:_indie]",low,neutral


In [190]:
def ohe_prep(df, column, new_name): 
    # create one hot encoded features of a specific column
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [191]:
subject_ohe = ohe_prep(sentiment, 'subjectivity','subject')
subject_ohe.iloc[0]

subject|high      0
subject|low       1
subject|medium    0
Name: 0, dtype: uint8

In [192]:
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(playlist_df['genres_list'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
genre_df.drop(columns='genre|unknown')
genre_df.reset_index(drop = True, inplace=True)
genre_df.iloc[0]

genre|_indie                     0.433185
genre|alt_z                      0.548112
genre|alternative_rock           0.000000
genre|ambient_pop                0.000000
genre|art_pop                    0.000000
genre|austindie                  0.000000
genre|australian_rock            0.000000
genre|bedroom_pop                0.569452
genre|boy_band                   0.000000
genre|dream_pop                  0.000000
genre|el_paso_indie              0.000000
genre|electropop                 0.000000
genre|etherpop                   0.000000
genre|folk                       0.000000
genre|gauze_pop                  0.000000
genre|indie_pop                  0.000000
genre|indie_poptimism            0.000000
genre|indie_rock                 0.000000
genre|indietronica               0.000000
genre|korean_ost                 0.000000
genre|korean_superband           0.000000
genre|la_indie                   0.000000
genre|la_pop                     0.000000
genre|metropopolis               0

In [193]:
print(playlist_df['artist_pop'].describe())

count    100.000000
mean      71.990000
std       14.497994
min        0.000000
25%       64.000000
50%       74.000000
75%       80.250000
max       90.000000
Name: artist_pop, dtype: float64


In [194]:
pop = playlist_df[["artist_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
pop_scaled.head()

,artist_pop
0,0.622222
1,0.622222
2,0.622222
3,0.622222
4,0.622222


In [195]:
def create_feature_set(df, float_cols):
    # tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genres_list'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.drop(columns='genre|unknown') # drop unknown genre
    genre_df.reset_index(drop = True, inplace=True)
    
    # sentiment analysis
    df = sentiment_analysis(df, "track_name")

    # one-hot encoding
    subject_ohe = ohe_prep(df, 'subjectivity','subject') * 0.3
    polar_ohe = ohe_prep(df, 'polarity','polar') * 0.5
    key_ohe = ohe_prep(df, 'key','key') * 0.5
    mode_ohe = ohe_prep(df, 'mode','mode') * 0.5

    # normalization 
    # scale popularity column
    pop = df[["artist_pop","track_pop"]].reset_index(drop = True)
    scaler = MinMaxScaler()
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.2 

    # scale audio columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # concanenate all features
    final = pd.concat([genre_df, floats_scaled, pop_scaled, subject_ohe, polar_ohe, key_ohe, mode_ohe], axis = 1)
    
    final['track_id']=df['track_id'].values
    
    return final

In [196]:
float_cols = playlist_df.dtypes[playlist_df.dtypes == 'float64'].index.values
playlist_df.to_csv("../spotifyrecommendationsystem/allsong_data.csv", index = False)

# generate features
complete_feature_set = create_feature_set(playlist_df, float_cols=float_cols)
complete_feature_set.to_csv("../spotifyrecommendationsystem/complete_feature.csv", index = False)
complete_feature_set.head()

,genre|_indie,genre|alt_z,genre|alternative_rock,genre|ambient_pop,genre|art_pop,genre|austindie,genre|australian_rock,genre|bedroom_pop,genre|boy_band,genre|dream_pop,genre|el_paso_indie,genre|electropop,genre|etherpop,genre|folk,genre|gauze_pop,genre|indie_pop,genre|indie_poptimism,genre|indie_rock,genre|indietronica,genre|korean_ost,genre|korean_superband,genre|la_indie,genre|la_pop,genre|metropopolis,genre|modern_alternative_rock,genre|modern_rock,genre|nz_pop,genre|permanent_wave,genre|pop,genre|pop_boy_group,genre|pop_rock,genre|pov,genre|rock,genre|shimmer_pop,genre|shiver_pop,genre|shoegaze,genre|social_media_pop,genre|stomp_and_holler,genre|unknown,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,track_pop,subject|high,subject|low,subject|medium,polar|negative,polar|neutral,polar|positive,key|0,key|1,key|2,key|3,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,track_id
0,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132817,0.126272,0.151413,0.050469,0.061398,0.000002,0.020302,0.101014,0.029879,0.124444,0.000000,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0UMFm4ndFr3PH5VsVYnCNG
1,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157881,0.087293,0.176442,0.012684,0.104156,0.000000,0.014244,0.111796,0.090826,0.124444,0.131034,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,7a2IJHzw9WJjoknwdnCop0
2,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048579,0.088852,0.125970,0.009379,0.062637,0.003604,0.016246,0.019711,0.126485,0.124444,0.082759,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0GD2v2Gl24z7F6bLl4fn7M
3,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.154522,0.124490,0.163239,0.006163,0.038883,0.000064,0.026891,0.095407,0.064823,0.124444,0.121839,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,4J4rJEaVpgr2tHwX6Jd13H
4,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.139535,0.091525,0.143128,0.017061,0.027935,0.000747,0.065416,0.046452,0.090959,0.124444,0.126437,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,3WowsoNoWeMqTH2suWv9MW


In [197]:
playlistDF_test = pd.read_csv("../spotifyrecommendationsystem/test_data.csv")
playlistDF_test.head()

,Unnamed: 0,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,analysis_url,duration_ms,time_signature,artist_pop,genres,track_pop
0,0,Stephen Sanchez,Until I Found You,Until I Found You,0T5iIrXA4p5GsubkhuBIKV,0.539,0.5080,10,-6.050,1,0.0288,0.7270,0.000000,0.1740,0.2270,101.358,https://api.spotify.com/v1/tracks/0T5iIrXA4p5G...,https://api.spotify.com/v1/audio-analysis/0T5i...,177720,3,75,gen_z_singer-songwriter,83
1,1,Peter McPoland,Romeo & Juliet,Romeo & Juliet,1VyA8mbSyXOdJx2lguR8Uu,0.318,0.4760,0,-7.108,1,0.0441,0.4540,0.000016,0.5030,0.0807,160.737,https://api.spotify.com/v1/tracks/1VyA8mbSyXOd...,https://api.spotify.com/v1/audio-analysis/1VyA...,166137,4,56,pov:_indie,63
2,2,Montell Fish,Fall in Love with You.,Fall in Love with You.,31MQR9rOHRXyg4J01hRY1h,0.483,0.0394,4,-19.525,1,0.0537,0.9220,0.012500,0.1120,0.1460,73.984,https://api.spotify.com/v1/tracks/31MQR9rOHRXy...,https://api.spotify.com/v1/audio-analysis/31MQ...,132000,4,73,pittsburgh_rap,0
3,3,Ali Gatie,YOU,It's You,5DqdesEfbRyOlSS3Tf6c29,0.732,0.4630,11,-6.972,0,0.0287,0.3740,0.000000,0.1940,0.3970,95.971,https://api.spotify.com/v1/tracks/5DqdesEfbRyO...,https://api.spotify.com/v1/audio-analysis/5Dqd...,212607,4,67,canadian_hip_hop,80
4,4,Keane,Hopes And Fears,Somewhere Only We Know,0ll8uFnc0nANY35E0Lfxvg,0.453,0.6550,9,-4.220,1,0.0259,0.0794,0.000008,0.0849,0.3680,172.012,https://api.spotify.com/v1/tracks/0ll8uFnc0nAN...,https://api.spotify.com/v1/audio-analysis/0ll8...,236160,4,72,neo_mellow piano_rock pop_rock,77


In [198]:
def generate_playlist_feature(complete_feature_set, playlist_df):
    # find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['track_id'].isin(playlist_df['track_id'].values)]
    
    # find all non-playlist song features
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['track_id'].isin(playlist_df['track_id'].values)]
    complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = "track_id")
    return complete_feature_set_playlist_final.sum(axis = 0), complete_feature_set_nonplaylist

In [199]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlistDF_test)

In [200]:
complete_feature_set_nonplaylist.head()

,genre|_indie,genre|alt_z,genre|alternative_rock,genre|ambient_pop,genre|art_pop,genre|austindie,genre|australian_rock,genre|bedroom_pop,genre|boy_band,genre|dream_pop,genre|el_paso_indie,genre|electropop,genre|etherpop,genre|folk,genre|gauze_pop,genre|indie_pop,genre|indie_poptimism,genre|indie_rock,genre|indietronica,genre|korean_ost,genre|korean_superband,genre|la_indie,genre|la_pop,genre|metropopolis,genre|modern_alternative_rock,genre|modern_rock,genre|nz_pop,genre|permanent_wave,genre|pop,genre|pop_boy_group,genre|pop_rock,genre|pov,genre|rock,genre|shimmer_pop,genre|shiver_pop,genre|shoegaze,genre|social_media_pop,genre|stomp_and_holler,genre|unknown,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,track_pop,subject|high,subject|low,subject|medium,polar|negative,polar|neutral,polar|positive,key|0,key|1,key|2,key|3,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,track_id
0,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132817,0.126272,0.151413,0.050469,0.061398,0.000002,0.020302,0.101014,0.029879,0.124444,0.000000,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0UMFm4ndFr3PH5VsVYnCNG
1,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157881,0.087293,0.176442,0.012684,0.104156,0.000000,0.014244,0.111796,0.090826,0.124444,0.131034,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,7a2IJHzw9WJjoknwdnCop0
2,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048579,0.088852,0.125970,0.009379,0.062637,0.003604,0.016246,0.019711,0.126485,0.124444,0.082759,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0GD2v2Gl24z7F6bLl4fn7M
3,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.154522,0.124490,0.163239,0.006163,0.038883,0.000064,0.026891,0.095407,0.064823,0.124444,0.121839,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,4J4rJEaVpgr2tHwX6Jd13H
4,0.433185,0.548112,0.0,0.0,0.0,0.0,0.0,0.569452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.139535,0.091525,0.143128,0.017061,0.027935,0.000747,0.065416,0.046452,0.090959,0.124444,0.126437,0.0,0.3,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,3WowsoNoWeMqTH2suWv9MW


In [201]:
complete_feature_set_playlist_vector

genre|_indie              0.000000
genre|alt_z               0.000000
genre|alternative_rock    0.000000
genre|ambient_pop         0.500000
genre|art_pop             1.730205
                            ...   
key|9                     0.000000
key|10                    0.000000
key|11                    0.000000
mode|0                    0.500000
mode|1                    1.500000
Length: 70, dtype: float64

In [202]:
def generate_playlist_recos(df, features, nonplaylist_features):   

    # find cosine similarity between the playlist and the complete song set
    non_playlist_df = df[df['track_id'].isin(nonplaylist_features['track_id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('track_id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40

In [203]:
pd.set_option('display.max_columns', None)
recommend = generate_playlist_recos(playlist_df, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)
recommend

/var/folders/r8/w16jv_cd7yxf2kslz6zvffgr0000gn/T/ipykernel_4850/3236604007.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('track_id', axis = 1).values, features.values.reshape(1, -1))[:,0]


,artist,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity,sim
46,Lana Del Rey,Brooklyn Baby,1NZs6n6hl8UuMaX0UC0YTz,0.404,0.6640,1,-6.698,1,0.0407,0.5420,0.004020,0.1100,0.0949,107.747,90,art_pop pop,83,"[art_pop, pop]",low,neutral,0.801558
51,Lana Del Rey,The Blackest Day,45pI8224KKX9Gx5fiyvEAh,0.445,0.5220,2,-8.007,1,0.0331,0.3630,0.000487,0.0797,0.1620,107.752,90,art_pop pop,71,"[art_pop, pop]",low,neutral,0.798841
56,Lana Del Rey,Radio,45dAw6GXEsogcDF3NUgj3O,0.312,0.8410,2,-5.244,1,0.0846,0.2170,0.000024,0.0876,0.1850,75.112,90,art_pop pop,56,"[art_pop, pop]",low,neutral,0.792675
49,Lana Del Rey,Florida Kilos,0iQPnK3bi7O27c5T2HBTdr,0.266,0.5130,0,-7.852,1,0.0388,0.2800,0.000060,0.1100,0.5790,200.781,90,art_pop pop,68,"[art_pop, pop]",low,neutral,0.744116
43,Lana Del Rey,Love,2Kerz9H9IejzeIpjhDJoYG,0.527,0.3660,10,-10.943,1,0.0277,0.4870,0.002300,0.1100,0.2340,98.994,90,art_pop pop,71,"[art_pop, pop]",high,positive,0.716321
36,Lana Del Rey,Love song,5tyMJlMqaggzvuX7TtlrTe,0.384,0.0701,9,-16.512,1,0.0367,0.9660,0.000170,0.0805,0.2880,63.425,90,art_pop pop,70,"[art_pop, pop]",high,positive,0.711615
37,Lana Del Rey,Cinnamon Girl,2mdEsXPu8ZmkHRRtAdC09e,0.294,0.3370,6,-10.614,0,0.0360,0.8140,0.414000,0.1400,0.1350,91.781,90,art_pop pop,85,"[art_pop, pop]",low,neutral,0.704933
54,Lana Del Rey,Video Games,5by7gtiDrxe4n2qQQunL8S,0.390,0.2550,6,-9.676,0,0.0299,0.8060,0.000001,0.0887,0.1790,122.056,90,art_pop pop,55,"[art_pop, pop]",low,neutral,0.703694
58,Lana Del Rey,Summertime Sadness,4cKtn8Shw999egpwBmWQmp,0.565,0.6470,1,-6.789,0,0.0334,0.0456,0.000000,0.1210,0.2160,111.931,90,art_pop pop,52,"[art_pop, pop]",low,neutral,0.696507
57,Lana Del Rey,Carmen,0LKYAQdeUDxqqL4O6qUzsD,0.487,0.4260,1,-9.560,0,0.0317,0.0879,0.000392,0.1280,0.0376,100.056,90,art_pop pop,49,"[art_pop, pop]",low,neutral,0.694145
